In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [54]:
df=pd.read_csv('netflix_titles.csv')

In [55]:
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [56]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


Feature Selecting Phase

In [57]:
df['Serial_No'] = range(1, len(df) + 1)

In [58]:
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Serial_No
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",1
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",2
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,3
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",4
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a...",8803
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g...",8804
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...,8805
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero...",8806


In [59]:
features =['type','listed_in','description','cast','rating','title']
print(features)

['type', 'listed_in', 'description', 'cast', 'rating', 'title']


In [60]:
for feature in features:
  df[feature]=df[feature].fillna('')

In [61]:
combine_features = df['type']+' '+df['listed_in']+' '+df['description']+' '+df['cast']+' '+df['rating']+' '+df['title']

In [62]:
print(combine_features)

0       Movie Documentaries As her father nears the en...
1       TV Show International TV Shows, TV Dramas, TV ...
2       TV Show Crime TV Shows, International TV Shows...
3       TV Show Docuseries, Reality TV Feuds, flirtati...
4       TV Show International TV Shows, Romantic TV Sh...
                              ...                        
8802    Movie Cult Movies, Dramas, Thrillers A politic...
8803    TV Show Kids' TV, Korean TV Shows, TV Comedies...
8804    Movie Comedies, Horror Movies Looking to survi...
8805    Movie Children & Family Movies, Comedies Dragg...
8806    Movie Dramas, International Movies, Music & Mu...
Length: 8807, dtype: object


Converting text to numerical values

In [63]:
vectorizer = TfidfVectorizer()

In [64]:
feature_vectors = vectorizer.fit_transform(combine_features)
print(feature_vectors)

  (0, 11075)	0.1853717941912493
  (0, 21515)	0.09955407202246529
  (0, 11935)	0.24485086591343636
  (0, 20)	0.12377682054577645
  (0, 34210)	0.08637051547705585
  (0, 21056)	0.28952217885823534
  (0, 14830)	0.17092307579493435
  (0, 6225)	0.1788669519726051
  (0, 44961)	0.13986345909981474
  (0, 19396)	0.14123488057265124
  (0, 45393)	0.05045634741557642
  (0, 48322)	0.1946419291983053
  (0, 9435)	0.26718652238583585
  (0, 2194)	0.049761696576081635
  (0, 21395)	0.26718652238583585
  (0, 20912)	0.05838478003349859
  (0, 11102)	0.15292898056515822
  (0, 42573)	0.23988358518840036
  (0, 22508)	0.3563626377100623
  (0, 24132)	0.23766039057721824
  (0, 15505)	0.19698649322162443
  (0, 25951)	0.1059056302112635
  (0, 19807)	0.1452498845703199
  (0, 32225)	0.053921660409029515
  (0, 13964)	0.1853717941912493
  :	:
  (8806, 14843)	0.15575278350325156
  (8806, 23470)	0.15150313335680887
  (8806, 519)	0.09744795194326794
  (8806, 6329)	0.11118664574787546
  (8806, 48561)	0.09663441680041968
  (

Getting similarity scores using cosine similarity

In [65]:
similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.01002447 0.02057067 ... 0.01071319 0.02041471 0.03099154]
 [0.01002447 1.         0.021469   ... 0.00101377 0.         0.00537259]
 [0.02057067 0.021469   1.         ... 0.00548834 0.01018153 0.03416644]
 ...
 [0.01071319 0.00101377 0.00548834 ... 1.         0.0574648  0.00595994]
 [0.02041471 0.         0.01018153 ... 0.0574648  1.         0.01013814]
 [0.03099154 0.00537259 0.03416644 ... 0.00595994 0.01013814 1.        ]]


In [66]:
movie_name = input('Enter the show name: ')

Enter the show name: money hiest


In [67]:
list_of_movies = df['title'].tolist()
print(list_of_movies)

['Dick Johnson Is Dead', 'Blood & Water', 'Ganglands', 'Jailbirds New Orleans', 'Kota Factory', 'Midnight Mass', 'My Little Pony: A New Generation', 'Sankofa', 'The Great British Baking Show', 'The Starling', 'Vendetta: Truth, Lies and The Mafia', 'Bangkok Breaking', 'Je Suis Karl', 'Confessions of an Invisible Girl', 'Crime Stories: India Detectives', 'Dear White People', "Europe's Most Dangerous Man: Otto Skorzeny in Spain", 'Falsa identidad', 'Intrusion', 'Jaguar', 'Monsters Inside: The 24 Faces of Billy Milligan', 'Resurrection: Ertugrul', 'Avvai Shanmughi', 'Go! Go! Cory Carson: Chrissy Takes the Wheel', 'Jeans', 'Love on the Spectrum', 'Minsara Kanavu', 'Grown Ups', 'Dark Skies', 'Paranoia', 'Ankahi Kahaniya', 'Chicago Party Aunt', 'Sex Education', 'Squid Game', 'Tayo and Little Wizards', 'The Father Who Moves Mountains', 'The Stronghold', 'Angry Birds', 'Birth of the Dragon', 'Chhota Bheem', 'He-Man and the Masters of the Universe', 'Jaws', 'Jaws 2', 'Jaws 3', 'Jaws: The Revenge

In [68]:
find_close_match = difflib.get_close_matches(movie_name,list_of_movies)
print(find_close_match)

['Johnny Test', 'Monkey Twins']


In [69]:
close_match = find_close_match[0]
print(close_match)

Johnny Test


Find the index of the movie

In [70]:
index_of_movie = df[df.title == close_match]['Serial_No'].values[0]
print(index_of_movie)

443


Getting similiar movies

In [71]:
similarity_score = list(enumerate(similarity[index_of_movie]))
print(similarity_score)

[(0, 0.018377517281727613), (1, 0.0210792458835131), (2, 0.008193547850178264), (3, 0.003087851669341271), (4, 0.00804944668015247), (5, 0.01832485298020186), (6, 0.003852441113054709), (7, 0.007267783011379308), (8, 0.005592735562968137), (9, 0.0050331262711837505), (10, 0.005333802816427646), (11, 0.0014243438121152264), (12, 0.012061242130770705), (13, 0.00636992975993922), (14, 0.02544702313650298), (15, 0.006483523038418507), (16, 0.006912507555971509), (17, 0.0499395857759357), (18, 0.0075805947963378615), (19, 0.003296706378103055), (20, 0.007998577881243474), (21, 0.0028746229237010448), (22, 0.009557400360277649), (23, 0.013910102207717587), (24, 0.013591449906743158), (25, 0.011461426385049241), (26, 0.008485693338990744), (27, 0.008428593846861081), (28, 0.004987390457138929), (29, 0.0050527673559935815), (30, 0.009127142121235816), (31, 0.0065702902290319205), (32, 0.006177727281028508), (33, 0.00227301970872895), (34, 0.00497323137743552), (35, 0.007078229641255107), (36, 

Sorting the list on similarity score

In [72]:
sorted_movies = sorted(similarity_score,key=lambda x:x[1],reverse=True)
print(sorted_movies)

[(443, 0.9999999999999999), (1460, 0.1400584946317762), (1840, 0.13802673495556028), (3001, 0.13429499234986297), (4167, 0.1331824361478083), (3711, 0.115933730469984), (5514, 0.11008768275172565), (3212, 0.10826844822429159), (4406, 0.10631152250495059), (7429, 0.10348221140382907), (2329, 0.10233463798723039), (3977, 0.10023866093501324), (2786, 0.09986331010168677), (7264, 0.09876993549804174), (5305, 0.09774741269280726), (6425, 0.09627371958323801), (4824, 0.09626700209137355), (2669, 0.09584070372221154), (4750, 0.09359771601407994), (3860, 0.09112519570280077), (8413, 0.0910242394797578), (2093, 0.09039988985175848), (5614, 0.089548896948557), (4767, 0.08942054465454666), (3046, 0.08876553186713017), (8050, 0.08241222118519267), (3575, 0.08238011960100335), (303, 0.08175635337123537), (3944, 0.08170132820144858), (6705, 0.08128355384165643), (3886, 0.08106472079002688), (3298, 0.08066760035855959), (1270, 0.08048526764521413), (7325, 0.0804742414566122), (2033, 0.079261486042439

In [73]:
print('Suggestions: \n')
i=1
for movie in sorted_movies:
  index=movie[0]
  title_from_index = df[df.index==index]['title'].values[0]
  if(i<16):
    print(i,'.',title_from_index)
    i+=1

Suggestions: 

1 . La ley de Herodes
2 . Monarca
3 . Someone Has to Die
4 . Playing with Fire
5 . Club of Crows
6 . Daniel Sosa: Maleducado
7 . Mar de Plástico
8 . La Reina del Sur
9 . The Queen of Flow
10 . Me and the Alien
11 . All For Love
12 . Superlopez
13 . A Life of Speed: The Juan Manuel Fangio Story
14 . Lady, la vendedora de rosas
15 . Narcos


Final interface

In [76]:
movie_name = input('Enter the show name: ')

list_of_movies = df['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name,list_of_movies)

close_match = find_close_match[0]

index_of_movie = df[df.title == close_match]['Serial_No'].values[0]

similarity_score = list(enumerate(similarity[index_of_movie]))

sorted_movies = sorted(similarity_score,key=lambda x:x[1],reverse=True)

print('Suggestions: \n')
i=1
for movie in sorted_movies:
  index=movie[0]
  title_from_index = df[df.index==index]['title'].values[0]
  if(i<16):
    print(i,'.',title_from_index)
    i+=1

Enter the show name: La Casa De Papel
Suggestions: 

1 . Money Heist: From Tokyo to Berlin
2 . Money Heist: The Phenomenon
3 . The Creative Indians
4 . Money, Explained
5 . The Great Heist
6 . London Heist
7 . Terrace House: Tokyo 2019-2020
8 . Time to Hunt
9 . Nuestra Lucha Libre
10 . Heist
11 . El Dragón: Return of a Warrior
12 . Haunted: Latin America
13 . Especial 20 años Fútbol de Primera
14 . Rica, Famosa, Latina
15 . Dirty Money
